In [ ]:
import requests
import pandas as pd
import time 
import json
from pandas.io.json import json_normalize  


#get auth token 

KEY = os.environ['PETFINDER_KEY']
SECRET = os.environ['PETFINDER_SECRET']

def get_header(self):
        data = {
            'grant_type': 'client_credentials',
            'client_id': KEY,
            'client_secret': SECRET
        }
        response = requests.post('https://api.petfinder.com/v2/oauth2/token', data=data)
        if response.reason != 'OK':
            print('failed to get token. check credentials')
            print('response code:', response.status_code)
            print('response reason:', response.reason)
        else:
            print('new token received')
        TOKEN = json.loads(response.text)['access_token']
        header = {
            'Authorization': 'Bearer {}'.format(TOKEN),
        }
        return header
    

#use token to get data 

type = 'Dog'
location = 60601    #60601 chicago  #63103 St L  #46202 Indy 
limit = 100
distance = 50  
status = 'adopted'
outfile = 'Chicago_adopted_20k_Jan_21.csv'

respdf = pd.DataFrame()

for i in range(200):
    page = i + 1 
    url = f'https://api.petfinder.com/v2/animals?type={type}&location={location}&limit={limit}&page={page}&distance={distance}&status={status}'
    print(i)
    response = requests.get(url, headers=header)
    my_dict =json.loads(response.text)
    animals = my_dict.get('animals')
    df = pd.io.json.json_normalize(animals)
    respdf = respdf.append(df)
    time.sleep(5)

respdf.to_csv(path_or_buf = outfile)

